# Homework-week3
## 1. 构建Seq2seq模型中的Encoder层，建议采用面向对象的编程形式。
部分代码参考, 尝试不同的单元结构，考虑加上预训练词向量
```python
class Encoder():
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        # code
    def call(self, x, hidden):
        # code
        pass

    def initialize_hidden_state(self):
        pass
```

## 2. 构建Seq2seq模型中的Dncoder层，建议采用面向对象的编程形式。
部分代码参考, 尝试不同的单元结构，考虑加上预训练词向量
```python
class Decoder():
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Decoder, self).__init__()
        # code
    def call(self, x, hidden, enc_output, context_vector):
        # code
        pass
```

## 3. 构建Seq2seq模型中的Attention层，建议采用面向对象的编程形式。<u>这部分这次作业选做</u>。
部分代码参考
```python
class BahdanauAttention():
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        # code

    def call(self, query, values):
        # hidden shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        # we are doing this to perform addition to calculate the score
        # code

        # score shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying score to self.V
        # the shape of the tensor before applying self.V is (batch_size, max_length, units)
        # code

        # attention_weights shape == (batch_size, max_length, 1)
        # code

        # context_vector shape after sum == (batch_size, hidden_size)
        # code

        return 
```





In [40]:
import sys
sys.path.append("../")

In [41]:
from utils import config

In [3]:
from gensim.models import word2vec

In [5]:
wv = word2vec.Word2Vec.load(config.w2v_model_path)

In [19]:
vocab = wv.wv.vocab

In [22]:
import pandas as pd

In [23]:
train_seg = pd.read_csv(config.train_seg_path)

In [26]:
train_seg.columns

Index(['QID', 'Brand', 'Model', 'Question', 'Dialogue', 'Report'], dtype='object')

In [31]:
train_seg.dropna(subset=['Question', 'Dialogue', 'Report'], how='any', inplace=True)

In [34]:
train_X = train_seg[['Question', 'Dialogue']].apply(lambda x: ' '.join(x), axis=1)
train_Y = train_seg[['Report']]

In [43]:
def pad_proc(sentence, max_len, vocab):
    '''
    < start > < end > < pad > < unk >
    '''
    # 0.按空格统计切分出词
    words = sentence.strip().split(' ')
    # 1. 截取规定长度的词数
    words = words[:max_len]
    # 2. 填充< unk > ,判断是否在vocab中, 不在填充 < unk >
    sentence = [word if word in vocab else '<UNK>' for word in words]
    # 3. 填充< start > < end >
    sentence = ['<START>'] + sentence + ['<STOP>']
    # 4. 判断长度，填充　< pad >
    sentence = sentence + ['<PAD>'] * (max_len + 2 - len(words))
    return ' '.join(sentence)

In [44]:
def get_max_len(data):
    """
    获得合适的最大长度值
    :param data: 待统计的数据  train_df['Question']
    :return: 最大长度值
    """
    max_lens = data.apply(lambda x: x.count(' '))
    return int(np.mean(max_lens) + 2 * np.std(max_lens))

In [47]:
max_len = get_max_len(train_X)

In [46]:
import numpy as np

In [48]:
max_len

248